<a href="https://colab.research.google.com/github/vaclavkratochvil/Consistency-independent-Multiple-criteria-Decision-Technique/blob/main/AHP_alternative_for_incosistent_cases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supplementary Material for **Multi-Criteria Decision Making Beyond Consistency: An Alternative to AHP for Real-World Industrial Problems**

This Jupyter Notebook contains the supplementary material for the paper *Multi-Criteria Decision Making Beyond Consistency: An Alternative to AHP for Real-World Industrial Problems* by *Silvia Carpitella*, *Václav Kratochvíl*, and *Miroslav Pištěk* submitted for publication in 2024. This document includes the source code and several examples demonstrating the usage and results discussed in the paper.


**!! To run the code, open it in [Google Colab](https://colab.research.google.com/github/vaclavkratochvil/AHP/blob/main/AHP_alternative_for_incosistent_cases.ipynb) !!**

The paper generalizes concepts from the recent publication:
>  Carpitella Silvia, Inuiguchi Masahiro, Kratochvíl Václav, Pištěk Miroslav :  Multi-criteria decision analysis without consistency in pairwise comparisons , Computers & Industrial Engineering vol.168, 2022. [10.1016/j.cie.2022.108089](https://doi.org/10.1016/j.cie.2022.108089)
---



## Aggregated Preference Matrix
The aggregated preference matrix is a composite matrix derived from multiple pairwise comparison matrices (PCMs) and a vector of weights that signify the relative importance of each PCM. Each reciprocal PCM $R^{(i)}$ is weighted by $w_i$​ from the weight vector $w$. The aggregated preference matrix is computed using the element-wise geometric mean of the weighted PCMs, ensuring that the results remains reciprocal. This process is crucial for combining evaluations across different criteria or experts into a single, coherent PCM for decision-making. For a precise definition, see equation (3) in refered paper.

In [ ]:
import numpy as np

def aggregated_preference_matrix(R_list, w):
    """
    Compute the aggregated preference matrix using element-wise weighted geometric mean.

    Parameters:
    R_list (list of np.ndarray): List of pairwise comparison matrices as NumPy arrays.
    w (np.ndarray): Weight vector.

    Returns:
    np.ndarray: Aggregated preference matrix.
    """
    # Ensure the number of matrices matches the number of weights
    if len(R_list) != len(w):
        raise ValueError("The number of matrices and weights must be the same.")

    # Compute the element-wise weighted geometric mean
    weighted_geom_mean = np.ones_like(R_list[0])
    for matrix, weight in zip(R_list, w):
        weighted_geom_mean *= np.power(matrix, weight)

    return weighted_geom_mean

# Define a helper function to print matrices or vectors with row and column names
def print_named_matrix_or_vector(matrix_or_vector, row_names, col_names=None):
    if col_names is None:  # It's a vector
        # Determine the maximum length of the names for proper alignment
        max_name_length = max(len(name) for name in row_names)

        # Print the vector
        for row_name, val in zip(row_names, matrix_or_vector):
            print(f"{row_name:>{max_name_length}}: {val:.4f}")
    else:  # It's a matrix
        # Determine the maximum length of the names for proper alignment
        max_name_length = max(max(len(name) for name in row_names), max(len(name) for name in col_names))

        # Print the column headers
        print(" " * (max_name_length + 1), end="")
        for name in col_names:
            print(f"{name:>{max_name_length}}", end=" ")
        print()

        # Print each row with its name
        for row_name, row in zip(row_names, matrix_or_vector):
            print(f"{row_name:>{max_name_length}}", end=" ")
            for val in row:
                print(f"{val:>{max_name_length}.4f}", end=" ")
            print()
        print()


## The canonical weight vector
The canonical weight vector is determined through a process that begins with a reciprocal pairwise comparison matrix (PCM). By taking the element-wise logarithm of the PCM, a skew-symmetric matrix is obtained. This matrix allows the formulation of a system of linear inequalities that identifies all maximal preferred elements based on the Subjective Stochastic Dominance (SSB) representation. To address any potential non-uniqueness among these elements, the optimal distribution of preference is defined as the unique maximal preferred element that maximizes entropy. This optimal distribution represents the most balanced and unbiased allocation of preferences.

The canonical weight vector is then derived by taking the normalized inverse of the vector resulting from multiplying the transpose of the reciprocal PCM by the optimal distribution of preference. This vector reflects the relative scores of individual alternatives, ensuring a coherent and consistent representation of preferences. For a consistent PCM, the canonical weight vector aligns with the weight vector used in the Analytic Hierarchy Process (AHP). This approach ensures that the derived weight vector accurately represents the underlying preferences and priorities, even in the presence of inconsistencies within the PCM.

The functions are based on equations (4), (5), and (6) in the referenced paper.

In [ ]:
import numpy as np
from scipy.optimize import minimize

def find_max_entropy_SSB_max(X):
    """
    It identifies the optimal distribution of preference within a given pairwise
    comparison matrix X. The function leverages the property that the element-wise
    logarithm of a PCM results in a skew-symmetric matrix, which allows the
    formulation of a system of linear inequalities to determine all maximal
    preferred elements in the sense of SSB (Subjective Stochastic Dominance) representation.

    To address potential non-uniqueness among these maximal preferred elements,
    the function defines the optimal distribution of preference as the unique
    maximal preferred element that maximizes entropy.

    Parameters:
    X (numpy.ndarray): Pairwise comparison matrix with preferences

    Returns:
    numpy.ndarray: The optimal distribution of preference that maximizes entropy.
    """

    # Shannon entropy as the objective function for minimization
    def shannon_entropy_point(point):
        probabilities = point / np.sum(point)
        entropy = -np.sum(probabilities * np.log(probabilities + 1e-9))  # Adding a small number to avoid log(0)
        return entropy

    # Automatically generate an initial point using the Dirichlet distribution
    def generate_initial_point(dimensions):
        return np.random.dirichlet(np.ones(dimensions))

    # Determine the number of dimensions from matrix X
    num_dimensions = X.shape[1]

    # Generate a valid initial point
    initial_point = generate_initial_point(num_dimensions)

    # Constraints for SSB maximal element
    constraints = [
        {'type': 'ineq', 'fun': lambda m: -np.dot(X, m)},  # Xm <= 0
        {'type': 'eq', 'fun': lambda m: np.sum(m) - 1},    # m1 + m2 + m3 + ... = 1
        {'type': 'ineq', 'fun': lambda m: m}               # m >= 0 (non-negative)
    ]

    # Find the maximum element (note the negative sign to maximize entropy)
    result = minimize(lambda m: -shannon_entropy_point(m), initial_point, constraints=constraints, bounds=[(0, None)] * num_dimensions)

    # Result
    return result.x

def kappa(R):
    """
    Compute the canonical weight vector \kappa(R) for reciprocal matrix R.

    Parameters:
    R (numpy.ndarray): Input reciprocal matrix.

    Returns:
    numpy.ndarray: weight vector of preferences.
    """
    # Step 1: Create SSB matrix X
    X = np.log(R)

    # Step 2: Find the minimum entropy point m_tilde
    m_tilde = find_max_entropy_SSB_max(X)

    # Step 3: Calculate vector pi
    pi = np.dot(R.T, m_tilde)

    # Step 4: Normalize vector 1/pi
    inv_pi = 1 / pi
    normalized_inv_pi = inv_pi / np.sum(inv_pi)

    return normalized_inv_pi


## Consistency independent Decision Technique
Given a multi-criteria decision-making problem encoded in AHP-like format. I.e Matrix $A$ represents the pairwise comparison matrix for the criteria relative to the goal. Each matrix in the list $B$ represents the pairwise comparison of alternatives with respect to each criterion.

Our consistency independent technique is the following:
1. Given reciprocal matrix $A$ we calculate the cannonical weight vector $\kappa(A)$
2. The obtained vector is used as a weight to aggregate matrices $B$ using the (weighted) geometric mean, thus obtaining aggregated preference matrix $B^{\kappa(A)}$
3. Such a matrix is again reciprocal, we may evaluate the vector of final weights $z_{CI}$ as $κ(B^{\kappa(A)})$






## Leader example
To demonstrate the typical problem, let's consider a decision-making scenario by [Wikipedia](https://w.wiki/AUJZ) involving three alternatives and four criteria. This can be exemplified using a scenario called "Tom, Dick, and Harry," which is based on a real-world situation of selecting a new leader for a company facing the retirement of its founder. In this example, there are three potential candidates for the leadership role - **Tom**, **Dick**, and **Harry** - and they are assessed based on four different criteria: **Age**, **Charisma**, **Education**, and **Experience**. The problem involves comparing these criteria in pairs and recording the preferences in a matrix form.

The example is discussed in **Section 4.3.1** in the referred paper.

First, we will define the pairwise comparison matrices for Experience, Education, Charisma, and Age, as well as the criteria comparison matrix.

In [ ]:
import numpy as np

# Leader names
leader_names = ["Tom", "Dick", "Harry"]

# Pairwise comparison matrix for Experience
experience = np.array([
    [1, 1/4, 4],
    [4, 1, 9],
    [1/4, 1/9, 1]
])
print("Experience:")
print_named_matrix_or_vector(experience, leader_names, leader_names)

# Pairwise comparison matrix for Education
education = np.array([
    [1, 3, 1/5],
    [1/3, 1, 1/7],
    [5, 7, 1]
])
print("Education:")
print_named_matrix_or_vector(education, leader_names, leader_names)

# Pairwise comparison matrix for Charisma
charisma = np.array([
    [1, 5, 9],
    [1/5, 1, 4],
    [1/9, 1/4, 1]
])
print("Charisma:")
print_named_matrix_or_vector(charisma, leader_names, leader_names)

# Pairwise comparison matrix for Age
age = np.array([
    [1, 1/3, 5],
    [3, 1, 9],
    [1/5, 1/9, 1]
])
print("Age:")
print_named_matrix_or_vector(age, leader_names, leader_names)

# List of pairwise comparison matrices
B_list = [experience, education, charisma, age]

# Criteria comparison matrix
criteria_names = ["Experience", "Education", "Charisma", "Age"]
criteria = np.array([
    [1, 4, 3, 7],
    [1/4, 1, 1/3, 3],
    [1/3, 3, 1, 5],
    [1/7, 1/3, 1/5, 1]
])
print("Criteria:")
print_named_matrix_or_vector(criteria, criteria_names, criteria_names)



Experience:
        Tom  Dick Harry 
  Tom 1.0000 0.2500 4.0000 
 Dick 4.0000 1.0000 9.0000 
Harry 0.2500 0.1111 1.0000 

Education:
        Tom  Dick Harry 
  Tom 1.0000 3.0000 0.2000 
 Dick 0.3333 1.0000 0.1429 
Harry 5.0000 7.0000 1.0000 

Charisma:
        Tom  Dick Harry 
  Tom 1.0000 5.0000 9.0000 
 Dick 0.2000 1.0000 4.0000 
Harry 0.1111 0.2500 1.0000 

Age:
        Tom  Dick Harry 
  Tom 1.0000 0.3333 5.0000 
 Dick 3.0000 1.0000 9.0000 
Harry 0.2000 0.1111 1.0000 

Criteria:
           Experience  Education   Charisma        Age 
Experience     1.0000     4.0000     3.0000     7.0000 
 Education     0.2500     1.0000     0.3333     3.0000 
  Charisma     0.3333     3.0000     1.0000     5.0000 
       Age     0.1429     0.3333     0.2000     1.0000 



Compute the result using our technique


In [ ]:
# Compute the cannonical weight vector for the criteria matrix using the kappa function
criteria_weights = kappa(criteria)

# Compute the aggregated preference matrix
aggregated_matrix = aggregated_preference_matrix(B_list, criteria_weights)

print("Aggregated Preference Matrix:")
print_named_matrix_or_vector(aggregated_matrix, leader_names, leader_names)

# Compute the final weights using the kappa function
final_weights = kappa(aggregated_matrix)

print("Final Weights:")
print_named_matrix_or_vector(final_weights, leader_names)

Aggregated Preference Matrix:
        Tom  Dick Harry 
  Tom 1.0000 0.6543 3.0879 
 Dick 1.5282 1.0000 4.2232 
Harry 0.3238 0.2368 1.0000 

Final Weights:
  Tom: 0.3460
 Dick: 0.5288
Harry: 0.1252


## Car example
The Jones family is in the market for a new vehicle. They have identified several criteria that are important to them in making their decision. These criteria include cost, safety, style, and capacity. Each criterion has specific subcriteria that further define the family's preferences.

**Criteria and Subcriteria**
* Cost:
  * Purchase Price: The initial cost of buying the car.
  * Maintenance Cost: The expected cost of maintaining the car.
  * Fuel Cost: The fuel efficiency and expected fuel expenses.
  * Resale Value: The estimated resale value of the car.

* Safety:
  The overall safety ratings and features of the car.

* Style:
  The aesthetic appeal and design of the car.

* Capacity:
  * Passenger Capacity: The number of passengers the car can comfortably accommodate.
* Cargo Capacity: The amount of cargo space available in the car.

**Alternatives:**
The family is considering the following car models:
 Accord Hybrid Sedan, Accord Sedan, CR-V SUV, Element SUV, Odyssey Minivan, Pilot SUV

See **Section 4.3.2** in the referred paper.


 Cost subcriteria:

In [ ]:
import numpy as np

# Car names
car_names = ["Accord Hybrid", "Accord Sedan", "CR-V", "Element", "Odyssey", "Pilot"]

# Purchase Price Matrix
purchase_price = np.array([
    [1, 1/9, 1/9, 1/9, 1/7, 1],
    [9, 1, 1, 1/2, 5, 9],
    [9, 1, 1, 1/2, 5, 9],
    [9, 2, 2, 1, 6, 9],
    [7, 1/5, 1/5, 1/6, 1, 7],
    [1, 1/9, 1/9, 1/9, 1/7, 1]
])

# Maintenance Cost Matrix
maintenance_cost = np.array([
    [1, 2/3, 4, 4, 5, 4],
    [3/2, 1, 4, 4, 5, 4],
    [1/4, 1/4, 1, 1, 3, 1],
    [1/4, 1/4, 1, 1, 2, 5/6],
    [1/5, 1/5, 1/3, 1/2, 1, 1],
    [1/4, 1/4, 1, 1.2, 1, 1]
])

# Resale Value Matrix
resale_value = np.array([
    [1, 1/3, 1/5, 1, 1, 2],
    [3, 1, 1/2, 2, 2, 4],
    [5, 2, 1, 4, 4, 6],
    [1, 2, 4, 1, 1, 2],
    [1, 2, 4, 1, 1, 2],
    [2, 4, 6, 2, 2, 1]
])

# Fuel Cost Matrix
fuel_cost = np.array([
    [1, 113/100, 13/10, 14/10, 135/100, 159/100],
    [100/113, 1, 23/20, 31/25, 119/100, 141/100],
    [10/13, 20/23, 1, 27/25, 26/25, 123/100],
    [5/7, 25/31, 25/27, 1, 25/26, 57/50],
    [20/27, 100/119, 25/26, 26/25, 1, 59/50],
    [100/159, 100/141, 100/123, 50/57, 50/59, 1]
])

# Cost Criteria Matrix
cost_criteria = np.array([
    [1, 2, 1/2, 2],
    [1/2, 1, 1/5, 1/2],
    [2, 5, 1, 3],
    [1/2, 2, 1/3, 1]
])

Capacity subcriteria:

In [ ]:
# Passenger Capacity Matrix
passenger_capacity = np.array([
    [1, 1, 5, 9, 6, 7],
    [1, 1, 5, 9, 6, 7],
    [1/5, 1/5, 1, 7, 5, 6],
    [1/9, 1/9, 1/7, 1, 1/5, 1/3],
    [1/6, 1/6, 1/5, 5, 1, 3],
    [1/7, 1/7, 1/6, 1/3, 1/3, 1]
])

# Cargo Capacity Matrix
cargo_capacity = np.array([
    [1, 1, 1/2, 1/2, 1/3, 1/2],
    [1, 1, 1/2, 1/2, 1/3, 1/2],
    [2, 2, 1, 1, 1/2, 1],
    [2, 2, 1, 1, 1/2, 1],
    [3, 3, 2, 2, 1, 2],
    [2, 2, 1, 1, 1/2, 1]
])

# Capacity Criteria Matrix
capacity_criteria = np.array([
    [1, 1/5],
    [5, 1]
])

Other Criteria Matrices:

In [ ]:
# Safety Matrix
safety = np.array([
    [1, 1, 7, 9, 1/3, 5],
    [1, 1, 7, 9, 1/3, 5],
    [1/7, 1/7, 1, 2, 1/8, 1/2],
    [1/9, 1/9, 1/2, 1, 1/9, 1/9],
    [3, 3, 8, 9, 1, 8],
    [1/5, 1/5, 1/2, 1, 1/8, 1]
])

# Style Matrix
style = np.array([
    [1, 1, 5, 9, 6, 7],
    [1, 1, 5, 9, 6, 7],
    [1/5, 1/5, 1, 7, 5, 6],
    [1/9, 1/9, 1/7, 1, 1/5, 1/3],
    [1/6, 1/6, 1/5, 5, 1, 3],
    [1/7, 1/7, 1/6, 1/3, 1/3, 1]
])

# Criteria Comparison Matrix
# in the following ordering: capacity, cost, safety, style
criteria = np.array([
    [1, 1/3, 1, 7],
    [3, 1, 3, 7],
    [1, 1/3, 1, 9],
    [1/7, 1/7, 1/9, 1]
])

Step-by-Step Solution:

 * **Subproblem 1: Cost Sub-Criteria Aggregation**
  Compute the aggregated preference matrix `cost` for the cost sub-criteria (Purchase Price, Maintenance Cost, Resale Value, Fuel Cost).

* **Subproblem 2: Capacity Sub-Criteria Aggregation**
  Compute the aggregated preference matrix `capacity` for the capacity sub-criteria (Passenger Capacity, Cargo Capacity).

* **Aggregate Main Criteria Matrices**
  Use the aggregated preference matrices from the subproblems as inputs for the main criteria (`cost, capacity, safety, style`).

In [ ]:

### Subproblem 1 ###
# compute the cannonical weight vector
weights = kappa(cost_criteria)
B_cost = [purchase_price, maintenance_cost, resale_value, fuel_cost]
cost = aggregated_preference_matrix(B_cost, weights)

print("Cost subproblem - aggregated matrix:")
print_named_matrix_or_vector(cost, car_names, car_names)

### Subproblem 2 ###
# compute the cannonical weight vector
weights = kappa(capacity_criteria)
B_capacity = [passenger_capacity, cargo_capacity]
capacity = aggregated_preference_matrix(B_capacity, weights)

print("Capacity subproblem - aggregated matrix:")
print_named_matrix_or_vector(capacity, car_names, car_names)

### Final solution ###
# compute the cannonical weight vector
weights = kappa(criteria)
B = [capacity, cost, safety, style]

# aggregate list of pairwise comparison matrices for capacity, cost, safety, style
aggregated_matrix = aggregated_preference_matrix(B, weights)

# evaluate the final vector
z_CI = kappa(aggregated_matrix)
print("Final results z_CI:")
print_named_matrix_or_vector(z_CI, car_names)

Cost subproblem - aggregated matrix:
              Accord Hybrid  Accord Sedan          CR-V       Element       Odyssey         Pilot 
Accord Hybrid        1.0000        0.3327        0.3159        0.7055        0.7626        1.7389 
 Accord Sedan        3.0055        1.0000        0.8339        1.4079        2.5180        4.1155 
         CR-V        3.1660        1.1992        1.0000        1.6887        3.2938        4.2863 
      Element        1.4173        1.4046        2.3155        1.0000        1.6526        2.4224 
      Odyssey        1.3113        0.7853        1.1871        0.6051        1.0000        2.3315 
        Pilot        1.1371        0.9501        1.3593        0.8163        0.8481        1.0000 

Capacity subproblem - aggregated matrix:
              Accord Hybrid  Accord Sedan          CR-V       Element       Odyssey         Pilot 
Accord Hybrid        1.0000        1.0000        0.7339        0.8094        0.5396        0.7762 
 Accord Sedan        1.0000   

## Industrial application
A textile manufacturing company in Italy seeks to optimize its supplier selection process using the Analytic Hierarchy Process (AHP) methodology to choose the best data analytics tool. The company's operations *span product development, manufacturing, sales, marketing, and overall management*. Key criteria for tool selection include *quality assurance, cost efficiency, delivery reliability, supplier reputation, and environmental impact*. Eight pre-screened tools are evaluated against these criteria. A decision-making group of three stakeholders (*supply chain analyst, market trend analyst, and sustainability auditor*) provides input, with their evaluations aggregated using AHP to ensure systematic, data-driven decision-making. Implementing the chosen tool involves significant costs in acquisition, infrastructure, training, and integration.

More information can be found in **Section 5** of the referred paper.

Definition:

In [ ]:
import numpy as np

# Define criteria names
cNames = ["C1", "C2", "C3", "C4", "C5"]

# Define the matrices DM1, DM2, DM3
DM1 = np.array([
    [1, 3, 2, 4, 4],
    [1/3, 1, 4, 5, 4],
    [1/2, 1/4, 1, 1/6, 5],
    [1/4, 1/5, 6, 1, 5],
    [1/4, 1/4, 1/5, 1/5, 1]
])

DM2 = np.array([
    [1, 3, 3, 4, 1],
    [1/3, 1, 1/5, 1/4, 1/2],
    [1/3, 5, 1, 2, 3],
    [1/4, 4, 1/2, 1, 4],
    [1, 2, 1/3, 1/4, 1]
])

DM3 = np.array([
    [1, 6, 6, 7, 1/8],
    [1/6, 1, 1/5, 1/4, 1/2],
    [1/6, 5, 1, 5, 1/3],
    [1/7, 4, 1/5, 1, 1/9],
    [8, 2, 3, 9, 1]
])

# Compute the geometric mean of the input matrices
DM = (DM1 * DM2 * DM3) ** (1/3)
print("Aggregated criteria matrix:" )
print_named_matrix_or_vector(DM, cNames, cNames)

# Define alternative names
bNames = ["AT1", "AT2", "AT3", "AT4", "AT5", "AT6", "AT7", "AT8"]

# Define the matrices B1, B2, B3, B4, B5
B1 = np.array([
    [1, 1/2, 1/8, 1/7, 2, 1/4, 1/6, 1/2],
    [2, 1, 1/9, 1/6, 3, 1/2, 1/2, 1/2],
    [8, 9, 1, 2, 9, 4, 3, 5],
    [7, 6, 1/2, 1, 8, 3, 2, 6],
    [1/2, 1/3, 1/9, 1/8, 1, 1/3, 1/6, 1/3],
    [4, 2, 1/4, 1/3, 3, 1, 2, 2],
    [6, 2, 1/3, 1/2, 6, 1/2, 1, 3],
    [2, 2, 1/5, 1/6, 3, 1/2, 1/3, 1]
])

B2 = np.array([
    [1, 1/2, 1/8, 1/7, 2, 1/4, 1/2, 1/2],
    [2, 1, 1/9, 1/6, 3, 1/2, 1/2, 1/2],
    [8, 9, 1, 2, 9, 4, 3, 5],
    [7, 6, 1/2, 1, 8, 3, 2, 2],
    [1/2, 1/3, 1/9, 1/8, 1, 1/3, 1/6, 1/3],
    [4, 2, 1/4, 1/3, 3, 1, 2, 2],
    [2, 2, 1/3, 1/2, 6, 1/2, 1, 1/6],
    [2, 2, 1/5, 1/2, 3, 1/2, 6, 1]
])

B3 = np.array([
    [1, 1/2, 1/8, 1/7, 2, 1/4, 1/2, 1/2],
    [2, 1, 1/9, 1/6, 3, 1/2, 1/2, 1/2],
    [8, 9, 1, 2, 9, 4, 3, 5],
    [7, 6, 1/2, 1, 8, 3, 2, 2],
    [1/2, 1/3, 1/9, 1/8, 1, 1/3, 1/6, 1/3],
    [4, 2, 1/4, 1/3, 3, 1, 2, 2],
    [2, 2, 1/3, 1/2, 6, 1/2, 1, 1/6],
    [2, 2, 1/5, 1/2, 3, 1/2, 6, 1]
])

B4 = np.array([
    [1, 1/2, 1/8, 1/7, 1/8, 1/4, 1/2, 1/2],
    [2, 1, 1/4, 1/6, 1/3, 2, 2, 2],
    [8, 4, 1, 2, 1/2, 4, 3, 5],
    [7, 6, 1/2, 1, 1/3, 3, 2, 2],
    [8, 3, 2, 3, 1, 6, 7, 8],
    [4, 1/2, 1/4, 1/3, 1/6, 1, 2, 2],
    [2, 1/2, 1/3, 1/2, 1/7, 1/2, 1, 1/6],
    [2, 1/2, 1/5, 1/2, 1/8, 1/2, 6, 1]
])

B5 = np.array([
    [1, 1/2, 1/8, 1/7, 2, 1/4, 1/2, 1/2],
    [2, 1, 1/4, 1/2, 5, 2, 2, 3],
    [8, 4, 1, 2, 8, 4, 3, 5],
    [7, 2, 1/2, 1, 7, 3, 2, 2],
    [1/2, 1/5, 1/8, 1/7, 1, 1/6, 1/7, 1/8],
    [4, 1/2, 1/4, 1/3, 6, 1, 2, 2],
    [2, 1/2, 1/3, 1/2, 7, 1/2, 1, 1/6],
    [2, 1/3, 1/5, 1/2, 8, 1/2, 6, 1]
])

# Store matrices in a dictionary for easy access
lB = [B1, B2, B3, B4, B5]

Aggregated criteria matrix:
   C1 C2 C3 C4 C5 
C1 1.0000 3.7798 3.3019 4.8203 0.7937 
C2 0.2646 1.0000 0.5429 0.6786 1.0000 
C3 0.3029 1.8420 1.0000 1.1856 1.7100 
C4 0.2075 1.4736 0.8434 1.0000 1.3050 
C5 1.2599 1.0000 0.5848 0.7663 1.0000 



Solution:

In [ ]:
weights = kappa(DM)
aggregated_matrix = aggregated_preference_matrix(lB, weights)
z_CI = kappa(aggregated_matrix)
print("Final results z_CI:")
print_named_matrix_or_vector(z_CI, bNames)

Final results z_CI:
AT1: 0.0457
AT2: 0.0564
AT3: 0.3656
AT4: 0.1828
AT5: 0.0630
AT6: 0.0914
AT7: 0.1219
AT8: 0.0731
